In [0]:
from numba import cuda, jit
import numpy as np

In [0]:
_SHA256_BLOCK_SIZE = 32

_ROTLEFT = lambda a, b : (((a) << (b)) | ((a) >> (32-(b))))
_ROTRIGHT = lambda a, b : (((a) >> (b)) | ((a) << (32-(b))))
_CH = lambda x, y, z : (((x) & (y)) ^ (~(x) & (z)))
_MAJ = lambda x, y, z : (((x) & (y)) ^ ((x) & (z)) ^ ((y) & (z)))
_EP0 = lambda x : (_ROTRIGHT(x,2) ^ _ROTRIGHT(x,13) ^ _ROTRIGHT(x,22))
_EP1 = lambda x : (_ROTRIGHT(x,6) ^ _ROTRIGHT(x,11) ^ _ROTRIGHT(x,25))
_SIG0 = lambda x : (_ROTRIGHT(x,7) ^ _ROTRIGHT(x,18) ^ ((x) >> 3))
_SIG1 = lambda x : (_ROTRIGHT(x,17) ^ _ROTRIGHT(x,19) ^ ((x) >> 10))

_k = np.array([ 0x428a2f98,0x71374491,0xb5c0fbcf,0xe9b5dba5,0x3956c25b,0x59f111f1,0x923f82a4,0xab1c5ed5,
            	0xd807aa98,0x12835b01,0x243185be,0x550c7dc3,0x72be5d74,0x80deb1fe,0x9bdc06a7,0xc19bf174,
        	    0xe49b69c1,0xefbe4786,0x0fc19dc6,0x240ca1cc,0x2de92c6f,0x4a7484aa,0x5cb0a9dc,0x76f988da,
                0x983e5152,0xa831c66d,0xb00327c8,0xbf597fc7,0xc6e00bf3,0xd5a79147,0x06ca6351,0x14292967,
                0x27b70a85,0x2e1b2138,0x4d2c6dfc,0x53380d13,0x650a7354,0x766a0abb,0x81c2c92e,0x92722c85,
                0xa2bfe8a1,0xa81a664b,0xc24b8b70,0xc76c51a3,0xd192e819,0xd6990624,0xf40e3585,0x106aa070,
                0x19a4c116,0x1e376c08,0x2748774c,0x34b0bcb5,0x391c0cb3,0x4ed8aa4a,0x5b9cca4f,0x682e6ff3,
                0x748f82ee,0x78a5636f,0x84c87814,0x8cc70208,0x90befffa,0xa4506ceb,0xbef9a3f7,0xc67178f2])

In [0]:
@cuda.jit(device=True)
def cuda_sha256_init(ctx_data, ctx_datalen, ctx_bitlen, ctx_state):
    ctx->datalen = 0
    ctx->bitlen = 0
    ctx->state[0] = 0x6a09e667
    ctx->state[1] = 0xbb67ae85
    ctx->state[2] = 0x3c6ef372
    ctx->state[3] = 0xa54ff53a
    ctx->state[4] = 0x510e527f
    ctx->state[5] = 0x9b05688c
    ctx->state[6] = 0x1f83d9ab
    ctx->state[7] = 0x5be0cd19

In [0]:
@cuda.jit(device=True)
def cuda_sha256_transform(ctx_state, data):
    m = np.zeros(64)
    i, j = 0, 0
    while i < 16:
        m[i] = (data[j] << 24) | (data[j+1] << 16) | (data[j+2] << 8) | (data[j+3])
        i += 1
        j += 4
    while i < 64:
        m[i] = _SIG1(m[i-2]) + m[i-7] + _SIG0(m[i-15]) + m[i-16]
        i += 1
    
    a, b, c, d, e, f, g, h = ctx_state

    i = 0
    while i < 64:
        t1 = h + _EP1(e) + _CH(e, f, g) + _k[i] + m[i]
        t2 = _EP0(a) + _MAJ(a, b, c)
        h = g
        g = f
        f = e
        e = d + t1
        d = c
        c = b
        b = a
        a = t1 + t2
        i += 1
    
    ctx_state[0] += a
    ctx_state[1] += b
    ctx_state[2] += c
    ctx_state[3] += d
    ctx_state[4] += e
    ctx_state[5] += f
    ctx_state[6] += g
    ctx_state[7] += h

In [0]:
@cuda.jit(device=True)
def cuda_sha256_update(ctx_data, ctx_datalen, ctx_bitlen, ctx_state, data, len):
    i = 0
    while i < 64:
        ctx_data[ctx_datalen] = data[i]
        ctx_datalen += 1
        if ctx_datalen == 64:
            cuda_sha256_transform(ctx_state, ctx_data)
            ctx_bitlen += 512
            ctx_datalen = 0

In [0]:
@cuda.jit(device=True)
def cuda_sha256_final(ctx_data, ctx_datalen, ctx_bitlen, ctx_state, hash):
    i = ctx_datalen
    
    if ctx_datalen < 56:
        ctx_data[i] = 0x80
        i += 1
        while i < 56:
            ctx_data[i] = 0x00
            i += 1
    else:
        ctx_data[i] = 0x80
        i += 1
        while i < 64:
            ctx_data[i] = 0x00
            i += 1
            cuda_sha256_transform(ctx_state, ctx_data)
            j = 0
            while j < 56:
                ctx_data[j] = 0x00
                j += 1

    ctx_bitlen += ctx_datalen * 8
    ctx_data[63] = ctx_bitlen
    ctx_data[62] = ctx_bitlen >> 8
    ctx_data[61] = ctx_bitlen >> 16
    ctx_data[60] = ctx_bitlen >> 24
    ctx_data[59] = ctx_bitlen >> 32
    ctx_data[58] = ctx_bitlen >> 40
    ctx_data[57] = ctx_bitlen >> 48
    ctx_data[56] = ctx_bitlen >> 56
    cuda_sha256_transform(ctx_state, ctx_data)

    i = 0
    while i < 4:
        hash[i] = (ctx_state[0] >> (24 - i * 8)) & 0x000000ff
        hash[i+4] = (ctx_state[1] >> (24 - i * 8)) & 0x000000ff
        hash[i+8] = (ctx_state[2] >> (24 - i * 8)) & 0x000000ff
        hash[i+12] = (ctx_state[3] >> (24 - i * 8)) & 0x000000ff
        hash[i+16] = (ctx_state[4] >> (24 - i * 8)) & 0x000000ff
        hash[i+20] = (ctx_state[5] >> (24 - i * 8)) & 0x000000ff
        hash[i+24] = (ctx_state[6] >> (24 - i * 8)) & 0x000000ff
        hash[i+28] = (ctx_state[7] >> (24 - i * 8)) & 0x000000ff
        i += 1

In [0]:
@cuda.jit
def kernel_sha256_hash(indata, inlen, outdata, outlen, n_batch):
    thread = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    if thread < n_batch:
        cuda_in = indata + thread * inlen
        cuda_out = outdata + thread * _SHA256_BLOCK_SIZE
        
        ctx_data = np.zeros(64)
        ctx_datalen = 0
        ctx_bitlen = 0
        ctx_state = np.zeroes(8)

        cuda_sha256_init(ctx_data, ctx_datalen, ctx_bitlen, ctx_state)
        cuda_sha256_update(ctx_data, ctx_datalen, ctx_bitlen, ctx_state, cuda_in, inlen)
        cuda_sha256_final(ctx_data, ctx_datalen, ctx_bitlen, ctx_state, cuda_out)

In [0]:
def mcm_cuda256_hash_batch(indata, inlen, outdata, outlen, n_batch):
    cuda_indata = cuda.todevice(indata)
    cuda_outdata = cuda.device_array(_SHA256_BLOCK_SIZE * n_batch)

    thread = 256
    block = (n_batch+thread-1) / thread

    kernel_sha256_hash[block, thread](cuda_indata, inlen, cuda_outdata, outlen, n_batch)
    cuda.synchronize()

    outdata = cuda_outdata.copy_to_host()